Send Reports as HL7 v2

In [31]:
import requests
import os
from dotenv import load_dotenv
from prompt_toolkit.shortcuts import clear

load_dotenv()

server = os.getenv("V2_SERVER")
file_list = ['ORU_R01_R125.1_R0A.txt','ORU_R01_R125.1_RBS.txt','ORU_R01_R125.1_REP.txt','ORU_R01_R125.1_RR8.txt','ORU_R01_R125.1_RX1.txt','ORU_R01_R125.1_SG9.txt','ORU_R01_R125.1_ZT001.txt','ORU_R01_R125.1_7A3.txt']


for file in file_list:
    with open('Examples/' + file, 'rb') as f:
        s = f.read()
        r = requests.post(server, data = s)
        print(r.status_code)
        print(r.text)

200
MSA|CA|ORIE-251014-9|ISC|IGENE|MFT|202601281656||ACK^R01|ORIE-251014-9|T|2.3
200
MSA|CA|ORIE-251014-9|ISC|IGENE|MFT|202601281656||ACK^R01|ORIE-251014-9|T|2.3


Convert v2 examples to FHIR Messages, and then send to FHIR Server

In [32]:
from requests.auth import HTTPBasicAuth

url = os.getenv("OAUTH2_TOKEN")
clientId = os.getenv("CLIENT_ID")
clientSecret = os.getenv("CLIENT_SECRET")

the_data = {"grant_type": "client_credentials", "scope": "system/*.*"}
headers = {'Content-Type': 'application/x-www-form-urlencoded'}
response = requests.post(url, auth=HTTPBasicAuth(clientId, clientSecret),verify=False,data=the_data, headers=headers)
responseInclude = response.json()
token = responseInclude['access_token']

print(token)
print("token was displayed")

/Users/kevinmayfield/Documents/GitHub/NHSNorthWestGMSA/Testing/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.20'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [37]:
import time

server = os.getenv("V2_TOOLS")
fhirServer = os.getenv("FHIR_SERVER")


headersFHIR = {'Content-Type': 'application/fhir+json',
           'Authorization': 'Bearer ' + token}
headersV2 = {'Content-Type': 'x-application/hl7-v2+er7'}
for file in file_list:
    with open('Examples/' + file, 'rb') as g:
        s = g.read()
        r = requests.post(server, data = s,verify=False,headers=headersV2)
        ##print(r.status_code)

        response = requests.post(fhirServer + '$process-message', verify=False,data=r.text, headers=headersFHIR)
        ##print(response.status_code)
        if response.status_code != 200:
            print(response.text)
            time.sleep(4)
            response = requests.post(fhirServer + '$process-message', verify=False,data=r.text, headers=headersFHIR)
            print(response.status_code)
            if response.status_code != 200:
                print("Retry Failed")


/Users/kevinmayfield/Documents/GitHub/NHSNorthWestGMSA/Testing/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.20'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
